# LangChain Overview

All code comes from [LangChain docs](langchain.readthedocs.io).

In [ ]:
!pip install langchain openai cohere huggingface_hub ipywidgets chromadb google-search-results Xformers tiktoken


In [ ]:
import os
import langchain
from langchain.llms import OpenAI, Cohere
from langchain import HuggingFacePipeline
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
os.environ["OPENAI_API_KEY"] = "TYPE YOUR API KEY"
os.environ["COHERE_API_KEY"] = "TYPE YOUR API KEY"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "TYPE YOUR API KEY"
os.environ["SERPAPI_API_KEY"] = "TYPE YOUR API KEY"

# LLMs

Integration with many LLM providers
- OpenAI
- Cohere
- AI21
- Huggingface Hub
- Azure OpenAI
- Manifest
- Goose AI
- Writer
- Banana
- Modal
- StochasticAI
- Cerebrium
- Petals
- Forefront AI
- PromptLayer OpenAI
- Anthropic
- DeepInfra
- Self-Hosted Models

In [ ]:
chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo')

In [ ]:
gpt3 = OpenAI(model_name='text-davinci-003')

In [ ]:
cohere = Cohere(model='command-xlarge')

In [ ]:
bloom = HuggingFacePipeline.from_model_id(model_id="bigscience/bloom-1b7", task="text-generation", model_kwargs={"temperature":0, "max_length":64})

In [ ]:
text = "How to be happy?"

In [ ]:
chatgpt([HumanMessage(content=text)])

In [ ]:
print(gpt3(text))

In [ ]:
print(cohere(text))

In [ ]:
print(bloom(text))

# Question Answering with external document

There are a lot of document loaders:
File Loader, Directory Loader, Notion, ReadTheDocs, HTML, PDF, PowerPoint, Email, GoogleDrive, Obsidian, Roam, EverNote, YouTube, Hacker News, GitBook, S3 File, S3 Directory, GCS File, GCS Directory, Web Base, IMSDb, AZLyrics, College Confidential, Gutenberg, Airbyte Json, CoNLL-U, iFixit, Notebook, Copypaste, CSV, Facebook Chat, Image, Markdown, SRT, Telegram, URL, Word Document, Blackboard

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('state_of_the_union.txt')

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
index.query(query)

# Keep memory

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory, CombinedMemory, ConversationSummaryMemory


conv_memory = ConversationBufferWindowMemory(
    memory_key="chat_history_lines",
    input_key="input",
    k=1
)

summary_memory = ConversationSummaryMemory(llm=OpenAI(), input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines"], template=_DEFAULT_TEMPLATE
)
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory,
    prompt=PROMPT
)

In [ ]:
conversation.run("Hi!")

In [ ]:
conversation.run("Can you tell me a joke?")

In [ ]:
conversation.run("Can you tell me a similar joke?")

# Chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9)
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

In [ ]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)

# Agent

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
tools = load_tools(["serpapi", "llm-math"], llm=gpt3)
agent = initialize_agent(tools, llm=gpt3, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")

In [ ]:
tools = load_tools(["serpapi", "python_repl"], llm=gpt3)
agent = initialize_agent(tools, llm=gpt3, agent="zero-shot-react-description", verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.49 power?")